# Regression -- Training a Tiny Fully-Connected NN on a simple function

In [ ]:
%load_ext autoreload
%autoreload 2

from loguru import logger
from tqdm import tqdm

from access_pytorch import config

Imports.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

Fix random seed for reproducibility.

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

Create datasets with `TensorDataset`, which is a wrapper around `Tensor` that allows us to use a tuple of tensors as a dataset.

In [ ]:
def generate_data(n_samples=1000):
    x = np.linspace(-1, 1, n_samples).reshape(-1, 1)
    y = np.sqrt(np.abs(x))
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

x_train, y_train = generate_data(1000)
x_test, y_test = generate_data(200)

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

A very very small FCNN.

In [ ]:
class tinyNN(nn.Module):
    def __init__(self, width=10, depth=2):
        super(tinyNN, self,).__init__()
        layers = [nn.Linear(1, width), nn.ReLU()] # input layer

        for _ in range(depth - 1): # hidden layers
            layers.append(nn.Linear(width, width))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(width, 1)) # output layer

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

Instantiation the model, we can play with `width` and `depth`.

In [ ]:
model = tinyNN(width=10, depth=1)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

The training loop, this is where the model `learns`.

In [ ]:
def train(model, train_loader, criterion, optimizer, epochs=500):
    model.train()
    for epoch in tqdm(range(epochs)):
        for x, y in train_loader:
            optimizer.zero_grad()
            pred = model(x)
            loss = loss_fn(pred, y)
            loss.backward()
            optimizer.step()

        if (epoch+1) % 50 == 0:
            logger.info(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

train(model, train_loader, loss_fn, optimizer)

Finished with training, we can look at the learned model and analyze test error.

In [ ]:
import torch.nn.functional as F

model.eval()
with torch.no_grad():
    pred = model(x_test).numpy()
    test_loss = F.mse_loss(model(x_test), y_test).item()

plt.plot(x_test, y_test, label='True')
plt.plot(x_test, pred, label='Predicted')
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.title(f'f(x) = sqrt(abs(x)) Test Loss: {test_loss:.6f}')